In [ ]:
import json    
import os
import torch as t
from torch import Tensor
from utils.display import draw_line_chart, draw_heatmap
import statistics

def get_file_names(directory):
    file_names = os.listdir(directory)
    return file_names


def compute_mean_std_keep_last_dim(tensor_list):
    """
    Compute the mean and standard deviation of a list of 1D tensors,
    resulting in 1D outputs of shape (length,).
    """
    # Stack into a tensor of shape [num_tensors, length]
    stacked = t.stack(tensor_list, dim=0)  # shape: (N, L)
    
    # Compute mean/std across dim=0 (the list dimension)
    mean_val = stacked.mean(dim=0)  # shape: (L,)
    std_val = stacked.std(dim=0)    # shape: (L,)
    
    return [mean_val.tolist(), std_val.tolist()]



def load_data_then_compute(
    task_name:str,
    directory:str,
    file_name: str,
) -> list[tuple[list, list]]:
    file_names = get_file_names(directory)

    data_0 = []
    data_1 = []
    data_2 = []
    for file in file_names:
        if task_name in file:
            data_dir = f'{directory}/{file}/{file_name}.json'
            with open(data_dir, "r") as f:
                data = json.load(f)
                tensor_data = Tensor(data).transpose(0,1)
                data_0.append(tensor_data[0].clone())
                data_1.append(tensor_data[1].clone())
                data_2.append(tensor_data[2].clone())
    result_0 = compute_mean_std_keep_last_dim(data_0)
    result_1 = compute_mean_std_keep_last_dim(data_1)
    result_2 = compute_mean_std_keep_last_dim(data_2)
    
    return [result_0, result_1, result_2]


def load_table_data_then_compute(
    task_name:str,
    directory:str,
    file_name: str,
) -> list[tuple[list, list]]:
    file_names = get_file_names(directory)

    data_0 = []
    data_1 = []
    for file in file_names:
        if task_name in file:
            data_dir = f'{directory}/{file}/{file_name}.json'
            with open(data_dir, "r") as f:
                data = json.load(f)
                tensor_data = Tensor(data)
                data_0.append(tensor_data[0].max().item())
                data_1.append(tensor_data[1].max().item())

    mean_s1 = statistics.mean(data_0)
    std_s1 = statistics.stdev(data_0) 
    mean_s2 = statistics.mean(data_1)
    std_s2 = statistics.stdev(data_1)

    return [mean_s1, std_s1, mean_s2, std_s2] 
    


In [ ]:
model_list = ['meta-llama/Llama-3.1-8B', 'meta-llama/Llama-3.2-3B', 'mistral-7b', 'Qwen/Qwen2.5-7B']


for model in model_list:

    directory = f'outputs/{model}/experiment_masking'

    task_list = []
    with open("data/list.json", "r") as f:
        data = json.load(f)
        for d in data:
            task_name = f"{d['task1']}-{d['task2']}"
            task_list.append(task_name)

    for i, task_name in enumerate(task_list):
        all_data = load_data_then_compute(task_name, directory, 'acc_results')
        draw_list = [
            [all_data[0][0], all_data[0][1], "$s_1 \\circ s_2$"],
            [all_data[1][0], all_data[1][1], "$s_1$"],
            [all_data[2][0], all_data[2][1], "$s_2$"],
        ]
        draw_line_chart(
            data=draw_list,
            x_name="Layer",
            log_path=f"figure/{directory}/{i}.svg",
            title=task_name,
            y_name="Accuracy",
            y_ticks= [0, 0.5, 1],
            colors=["firebrick", "royalblue", "seagreen"],    # optional
            linestyles=["-", "-", "-"],                 # optional
            show_grid=False,
            show_legend= True,
            dpi= 300,
            marker_size= 2,
            font_size= 10,
            tick_size= 10,
            figsize=(4,3)
        )

In [ ]:
model_list = ['meta-llama/Llama-3.1-8B', 'meta-llama/Llama-3.2-3B', 'mistral-7b', 'Qwen/Qwen2.5-7B']


for model in model_list:

    directory = f'outputs/{model}/experiment_masking_res'

    task_list = []
    with open("data/list.json", "r") as f:
        data = json.load(f)
        for d in data:
            task_name = f"{d['task1']}-{d['task2']}"
            task_list.append(task_name)

    for i, task_name in enumerate(task_list):
        all_data = load_data_then_compute(task_name, directory, 'ranking_results')
        draw_list = [
            [all_data[0][0], all_data[0][1], "$s_1 \\circ s_2$"],
            [all_data[1][0], all_data[1][1], "$s_1$"],
            [all_data[2][0], all_data[2][1], "$s_2$"],
        ]
        draw_line_chart(
            data=draw_list,
            x_name="Layer",
            log_path=f"figure/{directory}/{i}.svg",
            title=task_name,
            y_name="MRR",
            y_ticks= [0, 0.5, 1],
            colors=["firebrick", "royalblue", "seagreen"],    # optional
            linestyles=["-", "-", "-"],                 # optional
            show_grid=False,
            show_legend= True,
            dpi= 300,
            marker_size= 2,
            font_size= 10,
            tick_size= 10,
        )

In [ ]:
model_list = ['meta-llama/Llama-3.1-8B', 'meta-llama/Llama-3.2-3B', 'mistral-7b', 'Qwen/Qwen2.5-7B']


for model in model_list:

    directory = f'outputs/{model}/experiment_decoding_com'

    task_list = []
    with open("data/list.json", "r") as f:
        data = json.load(f)
        for d in data:
            task_name = f"{d['task1']}-{d['task2']}"
            task_list.append(task_name)

    for i, task_name in enumerate(task_list):
        all_data = load_data_then_compute(task_name, directory, 'ranking_results')
        attn_data = [all_data[0][0][0::2], all_data[1][0][0::2], all_data[2][0][0::2]]
        mlp_data = [all_data[0][0][1::2], all_data[1][0][1::2], all_data[2][0][1::2]]

        draw_heatmap(
            data= attn_data,
            row_labels=["$s_1 \\circ s_2$", "$s_1$", "$s_2$"],
            x_name= "Layer",
            title=task_name,
            log_path= f"figure/{directory}/attn_{i}.svg",
            cbar_label="MRR",
            cmap="Blues",
            figsize= (4, 1.5),
            dpi= 300,
            font_size= 10,
            tick_size= 10,
        )
        
        draw_heatmap(
            data= mlp_data,
            row_labels=["$s_1 \\circ s_2$", "$s_1$", "$s_2$"],
            x_name= "Layer",
            title=task_name,
            log_path= f"figure/{directory}/mlp_{i}.svg",
            cbar_label="MRR",
            cmap="Greens",
            figsize= (4, 3),
            dpi= 300,
            font_size= 10,
            tick_size= 10,
        )

In [ ]:
model_list = ['meta-llama/Llama-3.1-8B', 'meta-llama/Llama-3.2-3B', 'mistral-7b', 'Qwen/Qwen2.5-7B']

for model in model_list:
    directory = f'outputs/{model}/experiment_patching'

    # build task_list ---------------------------------------------------------
    with open("data/list.json", "r", encoding="utf-8") as f:
        task_list = [f"{d['task1']}-{d['task2']}" for d in json.load(f)]

    # collect results ---------------------------------------------------------
    save_results = {
        task: load_table_data_then_compute(task, directory, "strength_results")
        for task in task_list
    }

    # make sure the output folder exists --------------------------------------
    out_dir = f"figure/{directory}"
    os.makedirs(out_dir, exist_ok=True)

    # pretty-print JSON with newlines -----------------------------------------
    out_path = os.path.join(out_dir, "detect_subtask.json")
    with open(out_path, "w", encoding="utf-8") as outfile:
        json.dump(save_results, outfile, indent=2, ensure_ascii=False)  # <-- adds \n inside
        outfile.write("\n")  # <-- final trailing newline (optional)

In [ ]:
from src.data.dataset import load_data_with_instructions
from utils.display import  draw_line_chart
import json
import torch as t
import re
import ast


def get_evaluation_score(
    response: str,
)-> list:
    #matches = re.findall(r"\[[^\]]*\]", response)
    #if matches:
        #result = matches[-1]
        #lst = ast.literal_eval(result)
    result = response.split('[')[-1].split(']')[0]    
    lst = ast.literal_eval(result) 
    
    return lst
    

constraints_type = ["包含约束", "输出格式约束", "语气风格约束"]
data_list = load_data_with_instructions('data/2024_trace_evaluation.jsonl', constraints_type)
overall_score = t.zeros((4, 28))
n_layer = 28
evaluation_file = "outputs/trace_evaluation"


for layer in range(n_layer):
    eval_list = []
    with open(f"{evaluation_file}/layer{layer}.jsonl", "r", encoding="utf-8") as file:
        for line in file:
            json_obj = json.loads(line)
            eval_list.append(json_obj)

    data_num = 0
    s1_num = 0 
    s2_num = 0 
    s3_num = 0 
    for i in range(len(eval_list)):
        score_list = get_evaluation_score(eval_list[i])
        if not all(isinstance(item, int) for item in score_list):
            continue
        if len(score_list) != data_list[i]["constraints_num"]:
            continue
        
        data_num += 1
        for num_cons in range(data_list[i]["constraints_num"]):
            constraint = data_list[i]["levels2"][num_cons]
            if constraint == constraints_type[0]:
                s1_num += 1
                overall_score[0, layer] += score_list[num_cons] 
            elif constraint == constraints_type[1]:
                s2_num += 1
                overall_score[1, layer] += score_list[num_cons]
            elif constraint == constraints_type[2]:
                s3_num += 1
                overall_score[2, layer] += score_list[num_cons]
        overall_score[-1, layer] += sum(score_list)/len(score_list)

    overall_score[0, layer] /= s1_num
    overall_score[1, layer] /= s2_num
    overall_score[2, layer] /= s3_num
    overall_score[-1, layer] /= data_num


plot_list = [
    [overall_score[0].tolist(),[0]*n_layer ,"Inclusion"],
    [overall_score[1].tolist(),[0]*n_layer , "Output Format"],
    [overall_score[2].tolist(),[0]*n_layer , "Tone and Style"],
    [overall_score[-1].tolist(),[0]*n_layer , "Average"],
]


draw_line_chart(
            data=plot_list,
            x_name="Layer",
            log_path=f"figure/Trace.svg",
            y_name="Score",
            y_ticks= [0, 5, 10],
            colors=["royalblue", "seagreen", "orange","firebrick"],    # optional
            linestyles=["-", "-", "-", "-"],                 # optional
            show_grid=False,
            show_legend= True,
            dpi= 300,
            marker_size= 2,
            font_size= 10,
            tick_size= 10,
            figsize=(5,3.5)
        )